# **Features and Metafeatures' Extraction**

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import gymnasium as gym
from sklearn.preprocessing import LabelEncoder
from stable_baselines3 import PPO, A2C, DQN
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

## **Funções**

In [4]:
# Função para detectar o algoritmo com base no nome do arquivo
def detect_algorithm(folder, env_config_mapping):
    algorithm_name = env_config_mapping.get(folder, {}).get("model", None)
    
    algorithm_mapping = {
        "PPO": PPO,
        "A2C": A2C,
        "DQN": DQN
    }
    
    return algorithm_mapping.get(algorithm_name)

In [5]:
# Função para criar o ambiente e verificar o tipo de observação
def create_environment(folder, env_config_mapping, observation_type):
    env_config = env_config_mapping.get(folder, {})
    env = gym.make("highway-v0")
    env.unwrapped.configure(env_config)
    return env

In [6]:
def extract_features_from_model(model, env, n_episodes=100):
    # Listas para armazenar as métricas por episódio
    performance_data = []
    print("a")
    for episode in range(n_episodes):
        done = truncated = False
        print("b")
        obs, info = env.reset()
        episode_reward = 0
        time_steps = 0
        print("c")
        while not (done or truncated):
            print("d")
            print("Obs Shape na funcao extract: ", obs.shape)
            import inspect
            source_code = inspect.getsource(model.predict)
            print(source_code)
            action, _states = model.predict(obs, deterministic=True) # ERRO AQUI
            print("e")
            new_obs, reward, done, truncated, info = env.step(action)
            print("f")
            episode_reward += reward
            time_steps += 1
    
            # Armazenar informações de desempenho a cada passo
            performance_data.append({
                'Reward': reward,
                'Step': time_steps,
                'Speed': info.get('speed', np.nan),
                'Crashed': info.get('crashed', False),
                'Collision Reward': info.get('rewards', {}).get('collision_reward', 0),
                'Right Lane Reward': info.get('rewards', {}).get('right_lane_reward', 0),
                'High Speed Reward': info.get('rewards', {}).get('high_speed_reward', 0),
                'On Road Reward': info.get('rewards', {}).get('on_road_reward', 0),
            })

            obs = new_obs

    # Criação do DataFrame de desempenho
    performance_df = pd.DataFrame(performance_data)

    # Cálculo das métricas agregadas para o modelo
    average_reward = performance_df['Reward'].mean()
    reward_stddev = performance_df['Reward'].std()
    observation_variance = performance_df['Speed'].var()
    action_variance = performance_df['Reward'].var()
    total_transitions = len(performance_df)
    crash_rate = performance_df['Crashed'].mean()
    average_collision_reward = performance_df['Collision Reward'].mean()
    average_right_lane_reward = performance_df['Right Lane Reward'].mean()
    average_high_speed_reward = performance_df['High Speed Reward'].mean()
    average_on_road_reward = performance_df['On Road Reward'].mean()
    average_episode_length = performance_df['Step'].max()

    # Cálculo das novas métricas
    reward_sparsity = (performance_df['Reward'] != 0).mean()  # Proporção de recompensas não nulas
    transition_rewards_variance = performance_df['Reward'].var()

    # Obter o tipo de observação
    observation_type = env.unwrapped.config.get('observation', {}).get('type', 'Unknown')

    # Verificar se o tipo de observação é LidarObservation ou TimeToCollision e adicionar suas métricas
    max_range = None
    horizon = None
    if observation_type == "LidarObservation":
        max_range = env.unwrapped.config.get('observation', {}).get('max_range', 'N/A')
    elif observation_type == "TimeToCollision":
        horizon = env.unwrapped.config.get('observation', {}).get('horizon', 'N/A')

    # Obter Initial Lane Id e Ego Spacing
    initial_lane_id = env.unwrapped.config.get('initial_lane_id', 'N/A')
    ego_spacing = env.unwrapped.config.get('ego_spacing', 'N/A')

    # Obter o nome do algoritmo
    algorithm_name = model.__class__.__name__

    # Criar um dicionário com as informações do ambiente e as métricas agregadas
    env_config = {
        'Duration': env.unwrapped.config.get('duration', 'N/A'),
        'Number of Lanes': env.unwrapped.config.get('lanes_count', 'N/A'),
        'Number of Vehicles': env.unwrapped.config.get('vehicles_count', 'N/A'),
        'Vehicles Density': env.unwrapped.config.get('vehicles_density', 'N/A'),
        'Reward Speed Range': str(env.unwrapped.config.get('reward_speed_range', 'N/A')),
        'Scaling Factor': env.unwrapped.config.get('scaling', 'N/A'),
        'Algorithm': algorithm_name,
        'Average Reward': average_reward,
        'Reward Standard Deviation': reward_stddev,
        'Observation Variance': observation_variance,
        'Action Variance': action_variance,
        'Number of Transitions Observed': total_transitions,
        'Crash Rate': crash_rate,
        'Average Collision Reward': average_collision_reward,
        'Average Right Lane Reward': average_right_lane_reward,
        'Average High Speed Reward': average_high_speed_reward,
        'Average On Road Reward': average_on_road_reward,
        'Average Episode Length': average_episode_length,
        'Reward Sparsity': reward_sparsity,  # Sparsidade da recompensa
        'Transition Rewards Variance': transition_rewards_variance,  # Variância das recompensas
        'Observation Type': observation_type,  # Tipo de observação
        'Maximum Range (Lidar)': max_range,  # Alcance máximo (se for Lidar)
        'Horizon (TimeToCollision)': horizon,  # Horizonte (se for TimeToCollision)
        'Initial Lane Id': initial_lane_id,  # ID da faixa inicial
        'Ego Spacing': ego_spacing,  # Distância inicial do ego
    }

    # Transformar o dicionário em um DataFrame com uma única linha
    summary_df = pd.DataFrame([env_config])

    return {
        'summary_df': summary_df,
    }

## **Com Lidar e TTC - DÁ ERRO OBS SHAPE**

In [ ]:
# Caminho principal para os modelos
base_path = r'C:\Users\Caty\Documents\HighWay-Env\Highway-Env-Tests\HighWay'

# Carregar o mapeamento de ambientes e suas configurações
try:
    with open('environment_config.json', 'r') as f:
        env_config_mapping = json.load(f)
except FileNotFoundError:
    print("Arquivo 'environment_config.json' não encontrado.")
    env_config_mapping = {}

# Lista para armazenar dados de cada modelo
all_models_data = []

# Lista final de todos os DataFrames
all_summaries = []

# Caminho do arquivo txt onde serão salvos os caminhos dos modelos ignorados
ignored_models_file = 'ignored_models.txt'

# Carregar os caminhos dos modelos ignorados
try:
    with open(ignored_models_file, 'r') as f:
        ignored_model_paths = f.read().splitlines()  # Carregar cada linha (caminho) em uma lista
except FileNotFoundError:
    print(f"Arquivo '{ignored_models_file}' não encontrado.")
    ignored_model_paths = []

# Iterar sobre os caminhos dos modelos no arquivo 'ignored_models.txt'
for model_path in ignored_model_paths:
    try:
        # Extrair o nome da pasta a partir do caminho do modelo
        folder = os.path.basename(os.path.dirname(model_path))

        observation_type = env_config_mapping.get(folder, {}).get('observation', {}).get('type', 'Unknown')

        Algorithm = detect_algorithm(folder, env_config_mapping)
        
        if Algorithm is None:
            raise ValueError(f"Algoritmo não detectado para {folder}")
        
        model = Algorithm.load(model_path)
        
        # Criar o ambiente e verificar o tipo de observação
        env = create_environment(folder, env_config_mapping, observation_type)
        # print("wlwlwlekekjrkjhfe")
        
        model.set_env(env)


        # print("c")

        extracted_data = extract_features_from_model(model, env, n_episodes=100)
        print(extracted_data['summary_df'])

        if not extracted_data['summary_df'].empty:
            all_summaries.append(extracted_data['summary_df'])

    except Exception as e:
        print(f"Erro ao carregar modelo {model_path}: {e}")

# Concatenar os DataFrames de todos os modelos
if all_summaries:
    summary_df_combined = pd.concat(all_summaries, ignore_index=True)
    # Salvar ou processar o DataFrame combinado conforme necessário
    summary_df_combined.to_csv('summary_df_combined.csv', index=True)


## **Sem Lidar e TTC - FUNCIONA DIREITO, COMENTADO PARA NÃO CORRER O RISCO DE CORRER DUAS VEZES**

In [29]:
# # Caminho principal para os modelos
# base_path = r'C:\Users\Caty\Documents\HighWay-Env\Highway-Env-Tests\HighWay'

# # Carregar o mapeamento de ambientes e suas configurações
# try:
#     with open('environment_config.json', 'r') as f:
#         env_config_mapping = json.load(f)
# except FileNotFoundError:
#     print("Arquivo 'environment_config.json' não encontrado.")
#     env_config_mapping = {}

# # Lista para armazenar dados de cada modelo
# all_models_data = []

# # Lista final de todos os DataFrames
# all_summaries = []

# # Caminho do arquivo txt onde serão salvos os caminhos dos modelos ignorados
# ignored_models_file = 'ignored_models.txt'

# # Limpar o arquivo txt antes de começar a adicionar os caminhos
# with open(ignored_models_file, 'w') as f:
#     f.write("Modelos ignorados (LidarObservation ou TimeToCollision):\n")

# # Iterar sobre todas as pastas
# for folder in os.listdir(base_path):
#     folder_path = os.path.join(base_path, folder)

#     if os.path.isdir(folder_path):
#         for model_file in os.listdir(folder_path):
#             if model_file.endswith('.zip'):
#                 model_path = os.path.join(folder_path, model_file)

#                 try:
#                     observation_type = env_config_mapping.get(folder, {}).get('observation', {}).get('type', 'Unknown')
#                     # print(f"Observation type: {observation_type}")  

#                     # Ignorar modelos com observação LidarObservation ou TimeToCollision
#                     if observation_type in ['LidarObservation', 'TimeToCollision']:
#                         with open(ignored_models_file, 'a') as f:
#                             f.write(f"{model_path}\n")  # Escrever o caminho do modelo ignorado
#                         continue  # Pular para o próximo modelo

#                     Algorithm = detect_algorithm(folder, env_config_mapping)
                    
#                     if Algorithm is None:
#                         raise ValueError(f"Algoritmo não detectado para {folder}")
                    
#                     model = Algorithm.load(model_path)

#                     # Criar o ambiente e verificar o tipo de observação
#                     env = create_environment(folder, env_config_mapping)

#                     model.set_env(env)

#                     extracted_data = extract_features_from_model(model, env, n_episodes=100)
#                     print(extracted_data['summary_df'])
                    
#                     if not extracted_data['summary_df'].empty:
#                         all_summaries.append(extracted_data['summary_df'])

#                 except Exception as e:
#                     print(f"Erro ao carregar modelo {model_path}: {e}")

# # Concatenar os DataFrames de todos os modelos
# if all_summaries:
#     summary_df_combined = pd.concat(all_summaries, ignore_index=True)
#     # Salvar ou processar o DataFrame combinado conforme necessário
#     summary_df_combined.to_csv('summary_df_combined.csv', index=True)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
   Duration  Number of Lanes  Number of Vehicles  Vehicles Density  \
0        60                4                  50                 1   

  Reward Speed Range  Scaling Factor Algorithm  Average Reward  \
0           [20, 30]             5.5       PPO        0.754729   

   Reward Standard Deviation  Observation Variance  ...  \
0                   0.075884              6.111606  ...   

   Average High Speed Reward  Average On Road Reward  Average Episode Length  \
0                   0.125825                     1.0                      60   

   Reward Sparsity  Transition Rewards Variance  Observation Type  \
0              1.0                     0.005758        Kinematics   

   Maximum Range (Lidar)  Horizon (TimeToCollision)  Initial Lane Id  \
0                   None                       None             None   

   Ego Spacing  
0            2  

[1 rows x 25 columns]
Wrapping the env with a `Mon

In [30]:
summary_df_combined 
# DATASET VINDO DA EXTRAÇÃO SEM LIDAR E TTC

,Duration,Number of Lanes,Number of Vehicles,Vehicles Density,Reward Speed Range,Scaling Factor,Algorithm,Average Reward,Reward Standard Deviation,Observation Variance,...,Average High Speed Reward,Average On Road Reward,Average Episode Length,Reward Sparsity,Transition Rewards Variance,Observation Type,Maximum Range (Lidar),Horizon (TimeToCollision),Initial Lane Id,Ego Spacing
0,60,4,50,1.0,"[20, 30]",5.5,PPO,0.754729,0.075884,6.111606,...,0.125825,1.0,60,1.000000,0.005758,Kinematics,None,None,None,2.0
1,40,4,40,1.0,"[20, 30]",5.5,PPO,0.765053,0.089004,9.313594,...,0.171923,1.0,40,0.999175,0.007922,Kinematics,None,None,None,2.0
2,38,4,50,1.0,"[20, 30]",5.5,PPO,0.760916,0.086899,7.936457,...,0.154855,1.0,38,1.000000,0.007551,Kinematics,None,None,None,2.0
3,40,4,25,1.0,"[20, 30]",5.5,PPO,0.766873,0.094445,10.303575,...,0.196252,1.0,40,0.999732,0.008920,Kinematics,None,None,None,2.0
4,45,4,50,1.0,"[20, 30]",5.5,PPO,0.762661,0.091617,9.970945,...,0.184761,1.0,45,0.999065,0.008394,Kinematics,None,None,None,2.0
5,40,4,30,1.0,"[20, 30]",5.5,PPO,0.750377,0.083105,7.675548,...,0.143556,1.0,40,0.999476,0.006906,Kinematics,None,None,None,2.0
6,53,4,50,1.0,"[20, 30]",5.5,PPO,0.759000,0.075456,7.161883,...,0.140153,1.0,53,0.999604,0.005694,Kinematics,None,None,None,2.0
7,40,4,35,1.0,"[20, 30]",5.5,PPO,0.766752,0.080824,8.266295,...,0.171698,1.0,40,0.999209,0.006533,Kinematics,None,None,None,2.0
8,40,4,50,1.0,"[20, 30]",5.5,A2C,0.787108,0.142239,9.616851,...,0.278131,1.0,40,1.000000,0.020232,Kinematics,None,None,None,2.0
9,60,8,30,1.5,"[40, 50]",5.5,DQN,0.697054,0.043973,0.924906,...,0.000000,1.0,60,0.999622,0.001934,Kinematics,None,None,None,2.0


In [2]:
df = pd.read_csv('summary_df_combined.csv')

In [4]:
df.fillna("Not Applicable", inplace=True)

In [9]:
df = df.drop(df.columns[0], axis=1)

In [6]:
df.to_csv('summary_df_combined_1.csv', index=True)

In [12]:
df['Initial Lane Id'] = df['Initial Lane Id'].astype(str)

In [13]:
# PRE-PROCESSAMENTO SIMPLES - O ISA NÃO ACEITA VALORES NULOS NEM CATEGÓRICOS
colunas_nao_numericas = df.select_dtypes(include=['object', 'category']).columns


le = LabelEncoder()

for coluna in colunas_nao_numericas:
    df[coluna] = le.fit_transform(df[coluna])

df

,Number of Lanes,Number of Vehicles,Vehicles Density,Reward Speed Range,Scaling Factor,Algorithm,Average Reward,Reward Standard Deviation,Observation Variance,Action Variance,...,Average High Speed Reward,Average On Road Reward,Average Episode Length,Reward Sparsity,Transition Rewards Variance,Observation Type,Maximum Range (Lidar),Horizon (TimeToCollision),Initial Lane Id,Ego Spacing
0,4,50,1.0,0,5.5,2,0.754729,0.075884,6.111606,0.005758,...,0.125825,1.0,60,1.000000,0.005758,0,0,0,3,2.0
1,4,40,1.0,0,5.5,2,0.765053,0.089004,9.313594,0.007922,...,0.171923,1.0,40,0.999175,0.007922,0,0,0,3,2.0
2,4,50,1.0,0,5.5,2,0.760916,0.086899,7.936457,0.007551,...,0.154855,1.0,38,1.000000,0.007551,0,0,0,3,2.0
3,4,25,1.0,0,5.5,2,0.766873,0.094445,10.303575,0.008920,...,0.196252,1.0,40,0.999732,0.008920,0,0,0,3,2.0
4,4,50,1.0,0,5.5,2,0.762661,0.091617,9.970945,0.008394,...,0.184761,1.0,45,0.999065,0.008394,0,0,0,3,2.0
5,4,30,1.0,0,5.5,2,0.750377,0.083105,7.675548,0.006906,...,0.143556,1.0,40,0.999476,0.006906,0,0,0,3,2.0
6,4,50,1.0,0,5.5,2,0.759000,0.075456,7.161883,0.005694,...,0.140153,1.0,53,0.999604,0.005694,0,0,0,3,2.0
7,4,35,1.0,0,5.5,2,0.766752,0.080824,8.266295,0.006533,...,0.171698,1.0,40,0.999209,0.006533,0,0,0,3,2.0
8,4,50,1.0,0,5.5,0,0.787108,0.142239,9.616851,0.020232,...,0.278131,1.0,40,1.000000,0.020232,0,0,0,3,2.0
9,8,30,1.5,2,5.5,1,0.697054,0.043973,0.924906,0.001934,...,0.000000,1.0,60,0.999622,0.001934,0,0,0,3,2.0


In [14]:
df.to_csv('summary_df_combined_1.csv', index=True)